### Import relevent Libraries

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
def detect_and_predict_mask(frame, faceNet, maskNet):

    # Photoframe
    (h,w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0,(300,300),   # Converts the input frame (frame) into a "blob" to make it compatible with the neural network.
                                (104.0,177.0,123.0))

    # Capture / Crop -> Face & Location 
    # Sets the preprocessed image (blob) as input to the face detection network (faceNet)
    faceNet.setInput(blob)         # blob -> Input Image 
    detections = faceNet.forward()

    faces = []    # List of face images cropped from the frame
    locs = []      # List of face locations (bounding boxes)
    preds = []    # List of predictions from the facemask network

    # Loop over the detections
    detections.shape
    
    for i in range(0, detections.shape[2]):    # To detect how many faces in the frame
        confidence = detections[0,0,i,2]

    # If the face is captured from the frame
    
    if len(faces) > 0:
        faces = np.array(faces, dtype='float32')   # Convert matrix -> array 
        pred = maskNet.predict(faces, batch_size=32)    # maskNet is our trained model using which we are going to predict
        
        # For faster inference we are using batch_size predictions on all faces at a time rather one-by-one predictions in the above For Loop

    return(locs, preds)

### Construct the argument parser and parse the arguments

In [3]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-f", "--face", type=str,
    default="face_detector",
    help="path to face detector model directory")
ap.add_argument("-m", "--model", type=str,
    default="face_mask_model.h5",
    help="path to trained face mask detector model")
ap.add_argument("-c", "--confidence", type=float, default=0.5,
    help="minimum probability to filter weak detections")
args = vars(ap.parse_args())

### Load Pre-Trained Model - from OpenCV (cv2)

In [4]:
print("Face directory path:", args['face'])

Face directory path: C:\Users\priya\AppData\Roaming\jupyter\runtime\kernel-0d951486-5874-4fee-895d-e988eeacd07a.json


In [5]:
import os
import subprocess
from keras.models import load_model

# Step 1: Define the folder path
folder_path = r'C:\Priya learning path\Hope_Artifiacial_Intelligence\Deep Learning\Face Mask Detection Project\face_detector'

# Step 2: Remove the read-only attribute
try:
    subprocess.run(f'attrib -r "{folder_path}" /s /d', shell=True)
    print("Read-only attribute removed from the folder.")
except Exception as e:
    print(f"Failed to change folder properties: {e}")

# Step 3: Save the model
try:
    prototxtPath = os.path.join(folder_path,'deploy.prototxt')
    weightsPath = os.path.join(folder_path, 'res10_300x300_ssd_iter_140000.caffemodel')
    faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
    print("Successfully.")
except Exception as e:
    print(f"Failed to save the model: {e}")
    
# args['face'] -> Refers to the directory path containing the model files (deploy.prototxt and res10_300x300_ssd_iter_140000.caffemodel).
# prototxtPath: The path to the .prototxt file, which defines the architecture of the deep learning model (layers, connections, etc.).
# weightsPath: The path to the .caffemodel file contains the pre-trained weights for the model.
# cv2.dnn.readNet() returns a Net object representing the loaded model. This object can then be used for inference (e.g., detecting faces in an image).


Read-only attribute removed from the folder.
Successfully.


### Load Our Facemask detector Model

In [6]:
maskNet = load_model(args['model'])

In [7]:
# It supports video sources such as webcams, USB cameras, or video files

vs = VideoStream(src=0).start()   # 0 refers to the default webcam on your system.
time.sleep(2.0)                   # Introduces a delay of 2 seconds to give the camera hardware time to "warm up."

### Loop over the frames from the video stream

In [ ]:
while True:
    # grab the frame from the threaded video stream and resize it to have a maximum width of 400 pixels
    frame = vs.read() 
    frame = imutils.resize(frame, width=400)   # frame -> becomes a single image (in numpy array format) 
                                               # imutils -> which simplifies resizing operations
    
    # Detect faces in the frame and determine if they are wearing a face mask or not
    locs, preds = detect_and_predict_mask(frame, faceNet, maskNet)

    # Loop over the detected face locations and their corresponding locations
    for box, preds in zip(locs, preds):
        startX, startY, endX, endY = box     #unpack the bounding box and predictions
        with_mask, without_mask = pred

        # Determine the class label, box color & text color
        label = 'Mask' if with_mask > without_mask else 'NO Mask'
        color = (0,255,0) if label == 'Mask' else (0,0,255)    # OpenCV uses BGR (Blue, Green, Red) format instead of RGB
    
        # Display the label and bounding box on the output frame  
        if label == 'Mask':
            cv2.putText(frame, "Mask : You are allowed", (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2) 
        elif label == 'NO Mask':
            cv2.putText(frame, "No Mask : You are NOT allowed", (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2) 

    # Show the Output Frame 
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1) & 0xFF

    # If the `q` key was pressed, break from the loop
    if key == ord('q'):
        break

# Cleanup
cv2. destroyAllwindows()

vs.release()
